In [1]:
import pandas as pd
import cx_Oracle
from config import oracle_wfm_key
from config import oracle_ods_key
from sqlalchemy.types import String
from tqdm import tqdm
from datetime import datetime
import datetime as dt
import databases as db

In [2]:
ods = db.oracle('usrocu_support',oracle_ods_key)

start = dt.date.today() - dt.timedelta(days=60)
start = dt.date.today() - dt.timedelta(days=60)

# Construct the query using parameter binding
query = '''
    SELECT *
    FROM CCO_OBJECTS.QCC_REVIEW
    WHERE INSERT_USER = :insert_user AND SCORE_DATE >= :start_date
'''

# Execute the query and load the result into a DataFrame
aqm = pd.read_sql(query, ods, params={'insert_user': 'AQM IMPORT', 'start_date': start})

In [3]:
verint = db.ssms()
ee = '''select 
            EmployeeName,
            EmployeeID as employeeID,
            City, 
            StateName as State,
            Country
        from V_AdHoc_EmployeeContacts
        where (USERNAME = 'satverintfirstlogon'
            AND ContactMethod = 'PopupAddress')'''
ee_df = pd.read_sql(ee,verint)

In [4]:
ods = db.oracle('usrocu_support',oracle_ods_key)
location = f'''select
                "A"."AGENT_USER_ID" "usr",
                "A"."LOCATION"
            from "CCO_OBJECTS"."BPO_USER_LOCATION" "A"'''
location_df = pd.read_sql(location,ods)
location_df['usr'] = location_df['usr'].str.lower()
usr = f'''select distinct EmployeeID as employeeID,
                ExternalEmpIdent as usr
        from V_AdHoc_EmployeeDataSource
        where DataSourceID = 1'''
usr_df = pd.read_sql(usr,verint)
usr_df['usr'] = usr_df['usr'].str.lower()

In [5]:
employees = ee_df[['EmployeeName','employeeID']]
employees = employees.merge(usr_df, on='employeeID', how='left')
employees.head()

,EmployeeName,employeeID,usr
0,"Hunter, Liz",1,usresh
1,"Tarantino, Sarah",2,usrspo
2,"Test, Test",3,satverintwrkoptmgmt
3,"Propson, Julie",60,usranc
4,"Mikkelson, Greg",61,usrgih


In [6]:
# Group by 'coach_tso_user', 'agent_tso_user', and the month of 'score_date'
count_by_coach_agent_month = aqm.groupby(['coach_tso_user', pd.Grouper(key='score_date', freq='MS'), 'agent_tso_user'])['aqm_review_id'].count()

print(count_by_coach_agent_month)

coach_tso_user   score_date  agent_tso_user 
TELECOM\\USRNHZ  2023-05-01  TELECOM\\dmull1    6
                             TELECOM\\ncurr1    6
                             TELECOM\\nlevy1    7
                             TELECOM\\spalm2    8
                             TELECOM\\usrnpu    6
                                               ..
TELECOM\\usrzwz  2023-07-01  TELECOM\\usrqkp    4
                             TELECOM\\usrwqj    1
                             TELECOM\\usrwtn    5
                             TELECOM\\wblis1    4
                             TELECOM\\zburr1    4
Name: aqm_review_id, Length: 2382, dtype: int64


In [7]:
# Reset the index to convert the Series into a DataFrame
df_count = count_by_coach_agent_month.reset_index()

# Convert all string columns to lowercase
df_count = df_count.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

# Remove "telecom\\" from coach_tso_user and agent_tso_user columns
df_count['coach_tso_user'] = df_count['coach_tso_user'].str.replace(r'telecom\\\\', '')
df_count['agent_tso_user'] = df_count['agent_tso_user'].str.replace(r'telecom\\\\', '')

df_count['agent_usr'] = df_count['agent_tso_user']
df_count = df_count[['coach_tso_user', 'score_date', 'agent_tso_user', 'agent_usr',  'aqm_review_id']]
df_count.columns = ['coach','month','agent','agent_usr','aqm_evals']
print(df_count)

       coach      month   agent agent_usr  aqm_evals
0     usrnhz 2023-05-01  dmull1    dmull1          6
1     usrnhz 2023-05-01  ncurr1    ncurr1          6
2     usrnhz 2023-05-01  nlevy1    nlevy1          7
3     usrnhz 2023-05-01  spalm2    spalm2          8
4     usrnhz 2023-05-01  usrnpu    usrnpu          6
...      ...        ...     ...       ...        ...
2377  usrzwz 2023-07-01  usrqkp    usrqkp          4
2378  usrzwz 2023-07-01  usrwqj    usrwqj          1
2379  usrzwz 2023-07-01  usrwtn    usrwtn          5
2380  usrzwz 2023-07-01  wblis1    wblis1          4
2381  usrzwz 2023-07-01  zburr1    zburr1          4

[2382 rows x 5 columns]


C:\Users\usrocu\AppData\Local\Temp\ipykernel_4872\1188471978.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_count['coach_tso_user'] = df_count['coach_tso_user'].str.replace(r'telecom\\\\', '')
C:\Users\usrocu\AppData\Local\Temp\ipykernel_4872\1188471978.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_count['agent_tso_user'] = df_count['agent_tso_user'].str.replace(r'telecom\\\\', '')


In [8]:
merged_df = df_count.merge(employees[['EmployeeName', 'usr']], left_on='coach', right_on='usr', how='left')
df_count['coach'] = merged_df['EmployeeName']

merged_df = df_count.merge(employees[['EmployeeName', 'usr']], left_on='agent', right_on='usr', how='left')
df_count['agent'] = merged_df['EmployeeName']

In [9]:
print(df_count)

                 coach      month               agent agent_usr  aqm_evals
0     Atherton, Aneita 2023-05-01    Mullings, Dwayne    dmull1          6
1     Atherton, Aneita 2023-05-01    Currie, Nickesha    ncurr1          6
2     Atherton, Aneita 2023-05-01                 NaN    nlevy1          7
3     Atherton, Aneita 2023-05-01                 NaN    spalm2          8
4     Atherton, Aneita 2023-05-01    McDonald, Kadeem    usrnpu          6
...                ...        ...                 ...       ...        ...
2377    Baker, Pejouri 2023-07-01  Campbell, Shereika    usrqkp          4
2378    Baker, Pejouri 2023-07-01       Cooke, Janell    usrwqj          1
2379    Baker, Pejouri 2023-07-01    Pearson, Sheldon    usrwtn          5
2380    Baker, Pejouri 2023-07-01    Blisset, Whitney    wblis1          4
2381    Baker, Pejouri 2023-07-01      Burrey, Zakiya    zburr1          4

[2382 rows x 5 columns]


In [10]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
drop = f'''drop table "CCO_WFM"."AQM_Evals"'''
delete = f'''delete from "CCO_WFM"."AQM_Evals" where "MONTH" >= TO_DATE('{start.strftime('%Y-%m-%d')}', 'YYYY-MM-DD')'''
with ods as conn:
    conn.execute(delete)

In [11]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
cols = df_count.dtypes[df_count.dtypes=='object'].index
type_mapping = {col : String(100) for col in cols }
df_count.to_sql('AQM_Evals',con=ods,schema='CCO_WFM',if_exists='append',index=False,chunksize=10**4,dtype=type_mapping)

2382

In [12]:
df_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   coach      2340 non-null   object        
 1   month      2382 non-null   datetime64[ns]
 2   agent      2143 non-null   object        
 3   agent_usr  2382 non-null   object        
 4   aqm_evals  2382 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 93.2+ KB


In [13]:
df_count.loc[(df_count['coach'] == 'Lewis, Tishantay') & (df_count['month'] == '2023-07-01')]#['aqm_evals'].sum()

,coach,month,agent,agent_usr,aqm_evals
33,"Lewis, Tishantay",2023-07-01,"Bailey, Anecia",abail1,2
34,"Lewis, Tishantay",2023-07-01,"Joseph, Alickcia",ajose1,4
35,"Lewis, Tishantay",2023-07-01,"Clovis, Cheryl",cclov1,2
36,"Lewis, Tishantay",2023-07-01,NaN,iserv1,2
37,"Lewis, Tishantay",2023-07-01,"Louisy, Joella",jloui1,5
38,"Lewis, Tishantay",2023-07-01,"Montoute, Jamie",jmont1,2
39,"Lewis, Tishantay",2023-07-01,"Flavier, Nolan",nflav1,2
40,"Lewis, Tishantay",2023-07-01,"Taylor, Niyoka",ntayl1,2
41,"Lewis, Tishantay",2023-07-01,"Mathurin, Peter",pmath1,2
42,"Lewis, Tishantay",2023-07-01,"Poyser , Rayaun",rpoys1,2
